# Neural Networks

- 📺 **Video:** [https://youtu.be/DU_p-RBy5gM](https://youtu.be/DU_p-RBy5gM)

## Overview
Marks the beginning of the course's deep learning journey by introducing neural networks for NLP. This video likely starts with the basic intuition: a neural network is a series of linear transformations interwoven with non-linear activation functions, allowing models to fit much more complex relationships than a single linear classifier.

In [ ]:
import os, random
random.seed(0)
CI = os.environ.get('CI') == 'true'

## Key ideas
- It might use a simple example - classifying data points in 2D that are not linearly separable - and show that a network with a hidden layer can carve out non-linear decision boundaries to separate them (something a linear model cannot do).
- The video explains key concepts: neurons (units computing weighted sums), layers (input layer of features, hidden layer(s) that transform inputs, output layer giving final scores), and activation functions like ReLU or tanh that introduce non-linearity.
- It emphasizes that with at least one hidden layer, neural nets can learn more abstract features of the input.
- In NLP terms, instead of manually creating n-gram features, a neural network can learn its own feature representation from the data.

## Demo

In [ ]:
print('Try the exercises below and follow the linked materials.')

## Try it
- Modify the demo
- Add a tiny dataset or counter-example


## References
- [Eisenstein 4.2](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Multiclass lecture note](https://www.cs.utexas.edu/~gdurrett/courses/online-course/multiclass.pdf)
- [A large annotated corpus for learning natural language inference](https://www.aclweb.org/anthology/D15-1075/)
- [Authorship Attribution of Micro-Messages](https://www.aclweb.org/anthology/D13-1193/)
- [50 Years of Test (Un)fairness: Lessons for Machine Learning](https://arxiv.org/pdf/1811.10104.pdf)
- [[Article] Amazon scraps secret AI recruiting tool that showed bias against women](https://www.reuters.com/article/us-amazon-com-jobs-automation-insight/amazon-scraps-secret-ai-recruiting-tool-that-showed-bias-against-women-idUSKCN1MK08G)
- [[Blog] Neural Networks, Manifolds, and Topology](http://colah.github.io/posts/2014-03-NN-Manifolds-Topology/)
- [Eisenstein Chapter 3.1-3.3](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Dropout: a simple way to prevent neural networks from overfitting](https://dl.acm.org/doi/10.5555/2627435.2670313)
- [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)
- [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)
- [The Marginal Value of Adaptive Gradient Methods in Machine Learning](https://papers.nips.cc/paper/2017/hash/81b3833e2504647f9d794f7d7b9bf341-Abstract.html)


*Links only; we do not redistribute slides or papers.*